In [131]:
import pandas as pd
import dash
from dash import dcc, html, callback
import plotly.express as px
from dash.dependencies import Input, Output
import country_converter as cc

# dash.register_page(__name__, path='/gdp_economic', name="GDP and Economic Growth", order=3)

In [132]:
# data = pd.read_csv("/Users/prachikansal/Desktop/centrale med/mock project/GlobalImpact-Covid19/data/API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_31631.csv", sep=',', skiprows=4, header=None)
data = pd.read_csv("/Users/prachikansal/Desktop/centrale med/mock project/GlobalImpact-Covid19/data/gdp_data.csv", sep=',')
data = data.drop(data.index[-1:-6:-1]) # As the last 5 rows are blank and just contain null values, drop them
data 

# link to the dataset: https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.MKTP.CD&country=#

,Series Name,Series Code,Country Name,Country Code,2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,GDP (current US$),NY.GDP.MKTP.CD,Afghanistan,AFG,15856668555.8336,17805098206.3141,19907329777.5872,20146416757.5987,20497128555.6972,19134221644.7325,18116572395.0772,18753456497.8159,18053222687.4126,18799444490.1128,19955929052.1496,14266499429.8746,14502158192.0904,..
1,GDP (current US$),NY.GDP.MKTP.CD,Albania,ALB,11926926615.8015,12890760315.146,12319834195.331,12776224496.9718,13228147516.1168,11386853113.0189,11861199830.8396,13019726211.7369,15156424015.1977,15401826127.2539,15162734205.2462,17930565118.8176,18916378860.5488,22977677860.7979
2,GDP (current US$),NY.GDP.MKTP.CD,Algeria,DZA,177785053939.531,218331946925.304,227143746075.934,229701430292.157,238942664192.59,187493855609.345,180763839522.151,189880896903.073,194554483655.528,193459662090.677,164873415325.201,186265418570.697,225560256621.757,239899491127.742
3,GDP (current US$),NY.GDP.MKTP.CD,American Samoa,ASM,573000000,570000000,640000000,638000000,643000000,673000000,671000000,612000000,639000000,647000000,721000000,750000000,871000000,..
4,GDP (current US$),NY.GDP.MKTP.CD,Andorra,AND,3449925738.86069,3629133574.27783,3188652765.36184,3193512950.02419,3271685596.66321,2789881258.50362,2896610479.73077,3000162081.11976,3218419896.9641,3155149347.80639,2891001149.3611,3324683011.5903,3380602042.84283,3727673593.01922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,GDP (current US$),NY.GDP.MKTP.CD,Sub-Saharan Africa,SSF,1448107365068.66,1627672472377.66,1690475240154.75,1797507885741.07,1874275100667.03,1668662993704.66,1522011045162.78,1625855640496.81,1780908927291.34,1830460925279.6,1716220805506.51,1932765211586.67,2061102939319.23,2032749202553.06
262,GDP (current US$),NY.GDP.MKTP.CD,Sub-Saharan Africa (excluding high income),SSA,1447125748526.33,1626613553670.39,1689385832315.55,1796174725333.69,1872887522796.95,1667230590352.53,1520442531814.64,1624180269855.8,1779124613364.58,1828592235182.87,1714838253754.94,1931278037791.87,2059044970799.4,2030607752381.92
263,GDP (current US$),NY.GDP.MKTP.CD,Sub-Saharan Africa (IDA & IBRD countries),TSS,1448107365068.66,1627672472377.66,1690475240154.75,1797507885741.07,1874275100667.03,1668662993704.66,1522011045162.78,1625855640496.81,1780908927291.34,1830460925279.6,1716220805506.51,1932765211586.67,2061102939319.23,2032749202553.06
264,GDP (current US$),NY.GDP.MKTP.CD,Upper middle income,UMC,14630255984971,17496066354586.1,18674953139164.7,19833760666069.5,20590348415303.8,19827996310186,19905034053844,21837271216675.9,23304307154243.2,23700366929893.4,22969949394569.8,27373038582479.3,28567593041349.3,29403677430761.3


In [133]:
years = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2023"]
for year in years:
    data.rename(columns={f'{year} [YR{year}]': f'{year}'}, inplace=True)

pre_covid_years = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019"]
post_covid_years = ["2022","2023"]

data['Pre_Covid'] = data[pre_covid_years].apply(pd.to_numeric, errors = 'coerce').mean(axis=1)
data['Post_Covid'] = data[post_covid_years].apply(pd.to_numeric, errors = 'coerce').mean(axis=1)

selected_columns = ["Country Name","Pre_Covid","2020","2021","Post_Covid"]
filtered_data = data[selected_columns]

'''
The dataset contains GDP data for 268 countries, from the year 1960-2023. We only consider data from the past decade, so 2010-2023. 
2010-2019 is grouped as the Pre-Covid era.
2020, 2021 are visualized individually to study the pattern of GDP accurately
2022-2023 is grouped as the Post-Covid era.
'''


'\nThe dataset contains GDP data for 268 countries, from the year 1960-2023. We only consider data from the past decade, so 2010-2023. \n2010-2019 is grouped as the Pre-Covid era.\n2020, 2021 are visualized individually to study the pattern of GDP accurately\n2022-2023 is grouped as the Post-Covid era.\n'

In [134]:
filtered_data

,Country Name,Pre_Covid,2020,2021,Post_Covid
0,Afghanistan,1.870696e+10,19955929052.1496,14266499429.8746,1.450216e+10
1,Albania,1.299679e+10,15162734205.2462,17930565118.8176,2.094703e+10
2,Algeria,2.038058e+11,164873415325.201,186265418570.697,2.327299e+11
3,American Samoa,6.306000e+08,721000000,750000000,8.710000e+08
4,Andorra,3.179313e+09,2891001149.3611,3324683011.5903,3.554138e+09
...,...,...,...,...,...
261,Sub-Saharan Africa,1.686594e+12,1716220805506.51,1932765211586.67,2.046926e+12
262,Sub-Saharan Africa (excluding high income),1.685176e+12,1714838253754.94,1931278037791.87,2.044826e+12
263,Sub-Saharan Africa (IDA & IBRD countries),1.686594e+12,1716220805506.51,1932765211586.67,2.046926e+12
264,Upper middle income,1.998004e+13,22969949394569.8,27373038582479.3,2.898564e+13


In [135]:
@callback(Output("gdp_graph", "figure"), [Input("country", "value"), ])
def update_chart(dist_column):
    return create_gdp_line_chart(data)


In [136]:
def create_gdp_line_chart(data):
  fig = px.line(data, 
                x = 'Year',
                y = 'GDP',
                color='Country Name',
                title='Impact of Covid-19 on GDP',
                labels={'Year': 'Phase', 'GDP': 'GDP (in USD)'},
                markers=True)
  return fig

In [137]:
create_gdp_line_chart(melted_data)